# Content Enhancement DsPY実験

このノートブックでは、`content_enhancement.py`のプロンプトをDsPYで実験します。

## 実験内容
- Cornell Note生成プロンプトの最適化
- Quiz生成プロンプトの最適化
- プロンプトエンジニアリングの効果測定


In [16]:
# ライブラリのインポート
import os
import sys
from pathlib import Path

# プロジェクトルートをパスに追加
project_root = Path.cwd().parent
sys.path.append(str(project_root))

# DsPY関連
import dspy

# プロジェクト関連
from src.models import CornellNote, QuizItem, QuizOption, AnswerResult
from src.graphs.content_enhancement import (
    create_content_enhancement_graph,
    enhance_answer_content,
    cornell_note_generation_node,
    ContentEnhancementState
)
from src.config import get_openai_api_key_safe

print("✅ ライブラリのインポート完了")


✅ ライブラリのインポート完了


In [17]:
ContentEnhancementState


src.graphs.content_enhancement.ContentEnhancementState

In [15]:
cornell_note_generation_node


<function src.graphs.content_enhancement.cornell_note_generation_node(state: src.graphs.content_enhancement.ContentEnhancementState) -> src.graphs.content_enhancement.ContentEnhancementState>

In [11]:
# DsPYの設定
api_key = get_openai_api_key_safe()
if api_key:
    lm = dspy.OpenAI(model="gpt-4o-mini", api_key=api_key, max_tokens=2000)
    dspy.settings.configure(lm=lm)
    print("✅ DsPY設定完了 (OpenAI)")
else:
    print("⚠️ OpenAI API key not found, using mock LM")
    lm = dspy.LM(model="mock")
    dspy.settings.configure(lm=lm)


<function src.graphs.content_enhancement.cornell_note_generation_node(state: src.graphs.content_enhancement.ContentEnhancementState) -> src.graphs.content_enhancement.ContentEnhancementState>

## Cornell Note生成のDsPY実装


In [ ]:
# Cornell Note生成のDsPYクラス
class CornellNoteGenerator(dspy.Signature):
    """Cornell Note生成のためのDsPYシグネチャ"""

    question = dspy.InputField(desc="ユーザーの質問")
    answer_text = dspy.InputField(desc="回答テキスト")
    citations = dspy.InputField(desc="引用文献のリスト")

    cue = dspy.OutputField(desc="キーコンセプトと用語（1-2の短いフレーズ）")
    notes = dspy.OutputField(desc="論文のYAMLリスト、各論文にtitle（Markdownリンク）とpoints（2-4の簡潔な文）を含む")
    summary = dspy.OutputField(desc="全論文の1-2文の要約")


class CornellNoteModule(dspy.Module):
    """Cornell Note生成モジュール"""

    def __init__(self):
        super().__init__()
        self.generate = dspy.ChainOfThought(CornellNoteGenerator)

    def forward(self, question, answer_text, citations):
        # 引用文献を文字列に変換
        citation_list = "\n".join(f"- {c['title']} -> {c['link'].replace('http://','https://')}" for c in citations)

        return self.generate(
            question=question,
            answer_text=answer_text,
            citations=citation_list
        )

print("✅ Cornell Note生成モジュール定義完了")


## Quiz生成のDsPY実装


In [ ]:
# Quiz生成のDsPYクラス
class QuizGenerator(dspy.Signature):
    """Quiz生成のためのDsPYシグネチャ"""

    question = dspy.InputField(desc="ユーザーの質問")
    answer_text = dspy.InputField(desc="回答テキスト")

    quiz_questions = dspy.OutputField(desc="理解をテストするための2つの多肢選択問題（各問題に4つの選択肢と正解を含む）")


class QuizModule(dspy.Module):
    """Quiz生成モジュール"""

    def __init__(self):
        super().__init__()
        self.generate = dspy.ChainOfThought(QuizGenerator)

    def forward(self, question, answer_text):
        return self.generate(
            question=question,
            answer_text=answer_text
        )

print("✅ Quiz生成モジュール定義完了")


## サンプルデータの準備


In [ ]:
# サンプルデータの作成
sample_question = "機械学習における深層学習の役割は何ですか？"

sample_answer = """
深層学習は機械学習の一分野で、多層のニューラルネットワークを使用して複雑なパターンを学習します。
従来の機械学習手法と比較して、深層学習は大量のデータから自動的に特徴量を抽出し、
画像認識、自然言語処理、音声認識などの分野で優れた性能を発揮します。
特に畳み込みニューラルネットワーク（CNN）は画像処理に、リカレントニューラルネットワーク（RNN）は
時系列データや自然言語処理に広く使用されています。
"""

sample_citations = [
    {
        "title": "Deep Learning: A Comprehensive Overview",
        "link": "https://arxiv.org/abs/2101.00190",
        "authors": ["Yann LeCun", "Yoshua Bengio", "Geoffrey Hinton"]
    },
    {
        "title": "Convolutional Neural Networks for Visual Recognition",
        "link": "https://arxiv.org/abs/1409.1556",
        "authors": ["Alex Krizhevsky", "Ilya Sutskever", "Geoffrey Hinton"]
    },
    {
        "title": "Recurrent Neural Networks and LSTM: A Tutorial",
        "link": "https://arxiv.org/abs/1506.00019",
        "authors": ["Andrej Karpathy", "Justin Johnson", "Li Fei-Fei"]
    }
]

print("✅ サンプルデータ準備完了")
print(f"質問: {sample_question}")
print(f"回答の長さ: {len(sample_answer)} 文字")
print(f"引用文献数: {len(sample_citations)}")


## Cornell Note生成の実験


In [ ]:
# Cornell Note生成のテスト
cornell_module = CornellNoteModule()

print("🚀 Cornell Note生成を開始...")
try:
    result = cornell_module.forward(
        question=sample_question,
        answer_text=sample_answer,
        citations=sample_citations
    )

    print("✅ Cornell Note生成完了")
    print(f"CUE: {result.cue}")
    print(f"NOTES: {result.notes}")
    print(f"SUMMARY: {result.summary}")

except Exception as e:
    print(f"❌ Cornell Note生成エラー: {e}")
